![ML Logo](https://raw.githubusercontent.com/chicochica10/utad-spark-ml/master/images/utad-spark-ml.1x_Banner_300.png)
# **Lab de Regresión lineal**
#### Este lab cubre un pipeline común de aprendizaje supervisado, usando un subconjunto de [Million Song Dataset](http://labrosa.ee.columbia.edu/millionsong/) de [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD). Nuestro objetivo será entrenar un modelo de regresión lineal para predecir el año de una canción dado un conjunto de características de audio.
#### ** Este lab cubre: **
+  ####*Parte 1:* Leer y parsear el dataset inicial
 + #### *Visualización 1:* Características
 + #### *Visualización 2:* Cambiando etiquetas
+  ####*Parte 2:* Crear y evaluar un modelo baseline
 + #### *Visualización 3:* Predicho vs. real
+  ####*Parte 3:* Entrenamiento (via descenso del gradiente) y evaluación del modelo de regresión lineal
 + #### *Visualización 4:* Error de entrenamiento
+  ####*Parte 4:* Entrenamiento usando MLib y ajuste de hiperparámetros via búsqueda en grid
 + #### *Visualización 5:* Prediciones del mejor modelo
 + #### *Visualización 6:* Mapa de calor de hiperparámetros
+  ####*Parte 5:* Añadir interacciones entre características
 
####  Como referencia puedes mirar el api de Spark en  [API de Spark Python](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) y los métodos de NumPy en [Referencia NumPy](http://docs.scipy.org/doc/numpy/reference/index.html)

### ** Parte 1: Leer y parsear el dataset inicial **

#### ** (1a) Cargar y comprobar los datos **
#### Los datos en bruto están almacenados en un fichero de texto, empezaremos por cargarlos en un RDD, cada elemento de un RDD representa un "data point" reflejado como un string delimitado por comas. Cada string comienza con la etiqueta (label) que es un año seguido de unas características numéricas de audio. Utiliza el [método count](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.count) para comprobar cuantos "data points" tenemos. Usa el [método take](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take) para imprimir 5 "data points" en su formato inicial.

In [1]:
# carga de librería de testing
from test_helper import Test
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('utad-spark-ml', 'millionsong.txt')
fileName = os.path.join(baseDir, inputPath)

numPartitions = 2
rawData = sc.textFile(fileName, numPartitions)

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
numPoints = <RELLENA>
print numPoints
samplePoints = <RELLENA>
print samplePoints

In [ ]:
# TEST Carga y comprobación de los datos (1a)
Test.assertEquals(numPoints, 6724, 'valor incorrecto para numPoints')
Test.assertEquals(len(samplePoints), 5, 'longitud incorrecto para samplePoints')

#### ** (1b) Uso de  `LabeledPoint` **
#### En MLib las instancias de training etiquetadas se almacenan utilizando objetos [LabeledPoint](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint). Escribe la función de parsePoint que toma como entrada un punto de datos y lo parsea utilizando el método [unicode.split](https://docs.python.org/2/library/string.html#string.split) y devuelve un  `LabeledPoint`. Usa esta función para parsear samplePoints (del punto anterior). Imprime las características y la etiqueta para el primer training point utilizando los atributos `LabeledPoint.features` y `LabeledPoint.label`. Por último calcula el número de características para este dataset.

#### Nota: `split()` se puede llamar directemente desde un objeto  `unicode` o `str`. Por ejemplo, `u'split,me'.split(',')` devuelve `[u'split', u'me']`.

In [ ]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

# Ejemplo de un data point:
# '2001.0,0.884,0.610,0.600,0.474,0.247,0.357,0.344,0.33,0.600,0.425,0.60,0.419'
#  2001.0 es la etiqueta y el resto de valores son las características

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def parsePoint(line):
    """Conviere un unicode string con comas por separador en un `LabeledPoint`.

    Args:
        line (unicode): unicode string con comas por separador
        donde el primer elemento es la etiqueta y el resto son características 

    Returns:
        LabeledPoint: La linea convertida en un `LabeledPoint` que consiste en una etiqueta y características
    """
    <RELLENA>

parsedSamplePoints = <RELLENA>
firstPointFeatures = <RELLENA>
firstPointLabel = <RELLENA>
print firstPointFeatures, firstPointLabel

d = len(firstPointFeatures)
print d

In [ ]:
# TEST Usando LabeledPoint (1b)
Test.assertTrue(isinstance(firstPointLabel, float), 'La etiqueta debe ser un float')
expectedX0 = [0.8841,0.6105,0.6005,0.4747,0.2472,0.3573,0.3441,0.3396,0.6009,0.4257,0.6049,0.4192]
Test.assertTrue(np.allclose(expectedX0, firstPointFeatures, 1e-4, 1e-4),
                'características incorrectas para firstPointFeatures')
Test.assertTrue(np.allclose(2001.0, firstPointLabel), 'etiqueta incorrecta para firstPointLabel')
Test.assertTrue(d == 12, 'número de características incorrecto')

#### **Visualización 1: Características**
#### Primero cargaremos y arrancaremos la librería de visualizacion después tomaremos las características de 50 puntos generando un mapa de calor que visualiza cada característica en una escala de grises y muestra la variación de cada característica en esos 50 puntos de muestra. Las características estarán entre 0 y 1. Los valores más cercanos a 1 son más oscuros.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

sampleMorePoints = rawData.take(50)
# Puedes descomentar la línea de debajo para seleccionar características aleatorias cada vez que ejecutes la celda
# sampleMorePoints = rawData.takeSample(False, 50)

parsedSampleMorePoints = map(parsePoint, sampleMorePoints)
dataValues = map(lambda lp: lp.features.toArray(), parsedSampleMorePoints)

def preparePlot(xticks, yticks, figsize=(10.5, 6), hideLabels=False, gridColor='#999999',
                gridWidth=1.0):
    """Plantilla para generar el layout del plot."""
    plt.close()
    fig, ax = plt.subplots(figsize=figsize, facecolor='white', edgecolor='white')
    ax.axes.tick_params(labelcolor='#999999', labelsize='10')
    for axis, ticks in [(ax.get_xaxis(), xticks), (ax.get_yaxis(), yticks)]:
        axis.set_ticks_position('none')
        axis.set_ticks(ticks)
        axis.label.set_color('#999999')
        if hideLabels: axis.set_ticklabels([])
    plt.grid(color=gridColor, linewidth=gridWidth, linestyle='-')
    map(lambda position: ax.spines[position].set_visible(False), ['bottom', 'top', 'left', 'right'])
    return fig, ax

# generar el layout y plotear
fig, ax = preparePlot(np.arange(.5, 11, 1), np.arange(.5, 49, 1), figsize=(8,7), hideLabels=True,
                      gridColor='#eeeeee', gridWidth=1.1)
image = plt.imshow(dataValues,interpolation='nearest', aspect='auto', cmap=cm.Greys)
for x, y, s in zip(np.arange(-.125, 12, 1), np.repeat(-.75, 12), [str(x) for x in range(12)]):
    plt.text(x, y, s, color='#999999', size='10')
plt.text(4.7, -3, 'Feature', color='#999999', size='11'), ax.set_ylabel('Observation')
pass

#### **(1c) Encontrar el rango **
#### Vamos a examinar las etiquetas para encontrar el rango de los años de las canciones, para hacer esto parsea primero cada elemento del RDD `rawData` y después encuentra las etiquetas mayor y menor.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
parsedDataInit = rawData.map(<RELLENA>)
onlyLabels = parsedDataInit.map(<RELLENA>)
minYear = <RELLENA>
maxYear = <RELLENA>
print maxYear, minYear

In [ ]:
# TEST Encuentra el rango (1c)
Test.assertEquals(len(parsedDataInit.take(1)[0].features), 12,
                  'número de características incorrectas en el sample point')
sumFeatTwo = parsedDataInit.map(lambda lp: lp.features[2]).sum()
Test.assertTrue(np.allclose(sumFeatTwo, 3158.96224351), 'parsedDataInit tiene valores incorrectos')
yearRange = maxYear - minYear
Test.assertTrue(yearRange == 89, 'rango incorrecto entre minYear y maxYear')

#### **(1d) Cambiar etiquetas **
#### Como acabamos de ver las etiquetas están entre los años 1900 y los años 2000. En los problemas de ML es frecuente cambiar las etiquetas para que empiecen desde cero. Partiendo de  `parsedDataInit` crea un nuevo RDD de objetos  `LabeledPoint` de tal forma que la menor de las etiquetas sea igual a cero.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
parsedData = parsedDataInit.<RELLENA>

# Debería ser un LabeledPoint
print type(parsedData.take(1)[0])
# ver el primer punto
print '\n{0}'.format(parsedData.take(1))

In [ ]:
# TEST Cambio de etiquetas (1d)
oldSampleFeatures = parsedDataInit.take(1)[0].features
newSampleFeatures = parsedData.take(1)[0].features
Test.assertTrue(np.allclose(oldSampleFeatures, newSampleFeatures),
                'las nuevas característica no casan con las antiguas')
sumFeatTwo = parsedData.map(lambda lp: lp.features[2]).sum()
Test.assertTrue(np.allclose(sumFeatTwo, 3158.96224351), 'parsedData tiene valores incorrectos')
minYearNew = parsedData.map(lambda lp: lp.label).min()
maxYearNew = parsedData.map(lambda lp: lp.label).max()
Test.assertTrue(minYearNew == 0, 'año mínimo incorrecto en los datos cambiados')
Test.assertTrue(maxYearNew == 89, 'año máximo incorrecto en los datos cambiados')

#### ** Visualización 2: Cambio de etiquetas **
#### Vamos a echar un vistazo a las etiquetas antes y después de cambiarlas. Ambos scatter plots visualizan las tuplas que guardan i) una etiqueta y ii) el número de training point de esa etiqueta. El primer scatter plot usa las etiquetas iniciales mientras que el segundo usa las etiquetas cambiadas. Ambos plot debería ser iguales excepto por las etiquetas en el eje de las x's.

In [ ]:
# obtener los datos para el plot
oldData = (parsedDataInit
           .map(lambda lp: (lp.label, 1))
           .reduceByKey(lambda x, y: x + y)
           .collect())
x, y = zip(*oldData)

# generar el layout y hacer el plot
fig, ax = preparePlot(np.arange(1920, 2050, 20), np.arange(0, 150, 20))
plt.scatter(x, y, s=14**2, c='#d6ebf2', edgecolors='#8cbfd0', alpha=0.75)
ax.set_xlabel('Year'), ax.set_ylabel('Count')
pass

In [ ]:
# obtener los datos para el plot
newData = (parsedData
           .map(lambda lp: (lp.label, 1))
           .reduceByKey(lambda x, y: x + y)
           .collect())
x, y = zip(*newData)

# generar el layout y hacer el plot
fig, ax = preparePlot(np.arange(0, 120, 20), np.arange(0, 120, 20))
plt.scatter(x, y, s=14**2, c='#d6ebf2', edgecolors='#8cbfd0', alpha=0.75)
ax.set_xlabel('Year (shifted)'), ax.set_ylabel('Count')
pass

#### ** (1e) Training, validación y test sets **
#### Ya casi hemos terminado de paresear el dataset, nuestra última tarea será partirlo en los subconjuntos de training, validación y test.  Usa el  [métod randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) con la semilla y los pesos especificados para crear los RDDs que almacenen estos datasets. A continuación cachéalos  ya que los vamos a acceder muchas veces a lo largo del lab. Por último calcula el tamaño de cada dataset y verifica que la suma de sus tamaños es igual al valor calculado en la parte (1a).

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
weights = [.8, .1, .1]
seed = 42
parsedTrainData, parsedValData, parsedTestData = parsedData.<RELLENA>
parsedTrainData.<RELLENA>
parsedValData.<RELLENA>
parsedTestData.<RELLENA>
nTrain = parsedTrainData.<RELLENA>
nVal = parsedValData.<RELLENA>
nTest = parsedTestData.<RELLENA>

print nTrain, nVal, nTest, nTrain + nVal + nTest
print parsedData.count()

In [ ]:
# TEST Set de training, validación, y tests (1e)
Test.assertEquals(parsedTrainData.getNumPartitions(), numPartitions,
                  'número incorrecto de particiones para parsedTrainData')
Test.assertEquals(parsedValData.getNumPartitions(), numPartitions,
                  'número incorrecto de particiones para parsedValData')
Test.assertEquals(parsedTestData.getNumPartitions(), numPartitions,
                  'número incorrecto de particiones para parsedTestData')
Test.assertEquals(len(parsedTrainData.take(1)[0].features), 12,
                  'número incorrecto de características para parsedTrainData')
sumFeatTwo = (parsedTrainData
              .map(lambda lp: lp.features[2])
              .sum())
sumFeatThree = (parsedValData
                .map(lambda lp: lp.features[3])
                .reduce(lambda x, y: x + y))
sumFeatFour = (parsedTestData
               .map(lambda lp: lp.features[4])
               .reduce(lambda x, y: x + y))
Test.assertTrue(np.allclose([sumFeatTwo, sumFeatThree, sumFeatFour],
                            2526.87757656, 297.340394298, 184.235876654),
                'los datos parseados de  Train, Val, Test tienen valores incorrectos')
Test.assertTrue(nTrain + nVal + nTest == 6724, 'tamaño incorrecto para los datasets de Train, Val, Test')
Test.assertEquals(nTrain, 5371, 'valor incorrecto para nTrain')
Test.assertEquals(nVal, 682, 'valor incorrecto para nVal')
Test.assertEquals(nTest, 671, 'valor incorrecto para nTest')

### ** Parte 2: Crear y evaluar el modelo baseline**

#### **(2a) etiqueta media **
#### Un modelo muy simple pero natural que sirva como baseline es el que siempre devuelve la misma predicción independientemente del data point dado utilizando la etiqueta de la media aritmética en el set de training como valor de predicción constante. Calcula este esta media (cambiada) para los años de la canciones en el set de training. Busca en la  [API de los RDD](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) el método apropiado.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
averageTrainYear = (parsedTrainData
                    <RELLENA>)
print averageTrainYear

In [ ]:
# TEST etiqueta media aritmética (2a)
Test.assertTrue(np.allclose(averageTrainYear, 53.9316700801),
                'valor incorrecto para averageTrainYear')

#### **(2b) Error cuadrático medio **
#### Naturalmente queremos ver lo bien que se comporta este baseline simple. Para evaluarlo usaremos el error cuadrático medio ([RMSE](http://en.wikipedia.org/wiki/Root-mean-square_deviation)). Implementa la función que calcula el RMSE para tuplas de (label, prediction) y prueba esta función en un ejemplo.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def squaredError(label, prediction):
    """ Calcula el error cuadrático para una única predicción

    Args:
        label (float): El valor correcto para esta predicción.
        prediction (float): El valor predicho para esta observación.

    Returns:
        float: La diferencia entre  `label` y `prediction` al cuadrado.
    """
    <RELLENA>

def calcRMSE(labelsAndPreds):
    """ Calcula el error cuadrático medio para un `RDD` de tuplas (label, prediction).

    Args:
        labelsAndPred (RDD de (float, float)): un `RDD` de tuplas (label, prediction).

    Returns:
        float: El error cuadrático medio de los errores al cuadrado.
    """
    <RELLENA>

labelsAndPreds = sc.parallelize([(3., 1.), (1., 2.), (2., 2.)])
# RMSE = sqrt[((3-1)^2 + (1-2)^2 + (2-2)^2) / 3] = 1.291
exampleRMSE = calcRMSE(labelsAndPreds)
print exampleRMSE

In [ ]:
# TEST Error cuadrático medio (2b)
Test.assertTrue(np.allclose(squaredError(3, 1), 4.), 'definición incorrecta de squaredError')
Test.assertTrue(np.allclose(exampleRMSE, 1.29099444874), 'valor incorrecto para exampleRMSE')

#### **(2c) RMSE de training, validación y test  **
#### Calculemos ahora el RMSE de los sets de trainining, validación y test de nuestro modelo baseline. Para ello primero crearemos RDDs de tuplas  (label, prediction) para cada dataset y despúes llamaremos a calcRMSE. Nota que cada RMSE puede ser interpretado como el error de predicción medio para el dataset dado (en término de número de años).

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
labelsAndPredsTrain = parsedTrainData.<RELLENA>
rmseTrainBase = <RELLENA>

labelsAndPredsVal = parsedValData.<RELLENA>
rmseValBase = <RELLENA>

labelsAndPredsTest = parsedTestData.<RELLENA>
rmseTestBase = <RELLENA>

print 'Baseline Train RMSE = {0:.3f}'.format(rmseTrainBase)
print 'Baseline Validación RMSE = {0:.3f}'.format(rmseValBase)
print 'Baseline Test RMSE = {0:.3f}'.format(rmseTestBase)

In [ ]:
# TEST RMSE de Training, validación y test (2c)
Test.assertTrue(np.allclose([rmseTrainBase, rmseValBase, rmseTestBase],
                            [21.305869, 21.586452, 22.136957]), 'valor incorrecto para RMSE')

#### ** Visualización 3: Predichos vs. reales **
#### Visualizaremos las predicciones en el dataset de validación. Los scatter plots de abajo visualizan la tuplas que contienen el valor predicho y la etiqueta verdadera. El primer scatter plot representa la situación ideal donde  el valor predicho es el mismo que el de la etiqueta verdadera, mientras que el segundo plot usa la predicción baseline (`averageTrainYear`) para todos los valores predichos. Los puntos están codificados por color que van desde un amarillo claro cuando el valor predicho y el real son iguales hasta el rojo fuerte cuando son muy diferentes.

In [ ]:
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import get_cmap
cmap = get_cmap('YlOrRd')
norm = Normalize()

actual = np.asarray(parsedValData
                    .map(lambda lp: lp.label)
                    .collect())
error = np.asarray(parsedValData
                   .map(lambda lp: (lp.label, lp.label))
                   .map(lambda (l, p): squaredError(l, p))
                   .collect())
clrs = cmap(np.asarray(norm(error)))[:,0:3]

fig, ax = preparePlot(np.arange(0, 100, 20), np.arange(0, 100, 20))
plt.scatter(actual, actual, s=14**2, c=clrs, edgecolors='#888888', alpha=0.75, linewidths=0.5)
ax.set_xlabel('Predicted'), ax.set_ylabel('Actual')
pass

In [ ]:
predictions = np.asarray(parsedValData
                         .map(lambda lp: averageTrainYear)
                         .collect())
error = np.asarray(parsedValData
                   .map(lambda lp: (lp.label, averageTrainYear))
                   .map(lambda (l, p): squaredError(l, p))
                   .collect())
norm = Normalize()
clrs = cmap(np.asarray(norm(error)))[:,0:3]

fig, ax = preparePlot(np.arange(53.0, 55.0, 0.5), np.arange(0, 100, 20))
ax.set_xlim(53, 55)
plt.scatter(predictions, actual, s=14**2, c=clrs, edgecolors='#888888', alpha=0.75, linewidths=0.3)
ax.set_xlabel('Predicted'), ax.set_ylabel('Actual')

### ** Parte 3: Entrenamiento (vía descenso del gradiente) y evaluación de un modelo de regresión lineal **

#### ** (3a) sumando del gradiente **
#### Veamos ahora si podemos hacerlo mejor usando regresión lineal entrenando un modelo por el método vía descenso del gradiente (omitimos el interceptor por ahora). Recordemos que el cálculo la siguiente iteración por el método del descenso del gradiente en una regresión lineal es: $$ \scriptsize \mathbf{w}_{i+1} = \mathbf{w}_i - \alpha_i \sum_j (\mathbf{w}_i^\top\mathbf{x}_j  - y_j) \mathbf{x}_j \,.$$ donde $ \scriptsize i $ es el número de iteración del algoritmo de descenso de gradiente y $ \scriptsize j $ identifica la observación.
#### Primero implementa una función que calcula la suma en una iteración, esto es: $ \scriptsize (\mathbf{w}^\top \mathbf{x} - y) \mathbf{x} \, ,$ y prueba la función con dos ejemplos. Usa el método [dot](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.linalg.DenseVector.dot) de `DenseVector`.

In [ ]:
from pyspark.mllib.linalg import DenseVector

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def gradientSummand(weights, lp):
    """Calcula el sumando del gradiente para un peso y un `LabeledPoint`.

    Nota:
        `DenseVector` se comporta de una manera similiar a `numpy.ndarray` y se pueden intercambiar dentro
        de esta función. Por ejemplo ambos implementan el método `dot`.

    Args:
        weights (DenseVector): Un array de pesos del modelo (betas).
        lp (LabeledPoint): El `LabeledPoint` para una única observación.

    Returns:
        DenseVector: Un array de valores  con la misma longitud que `weights`.  El sumando del gradiente.
    """
    <RELLENA>

exampleW = DenseVector([1, 1, 1])
exampleLP = LabeledPoint(2.0, [3, 1, 4])
# gradientSummand = (dot([1 1 1], [3 1 4]) - 2) * [3 1 4] = (8 - 2) * [3 1 4] = [18 6 24]
summandOne = gradientSummand(exampleW, exampleLP)
print summandOne

exampleW = DenseVector([.24, 1.2, -1.4])
exampleLP = LabeledPoint(3.0, [-1.4, 4.2, 2.1])
summandTwo = gradientSummand(exampleW, exampleLP)
print summandTwo

In [ ]:
# TEST Sumando del gradiente (3a)
Test.assertTrue(np.allclose(summandOne, [18., 6., 24.]), 'valor incorrecto para summandOne')
Test.assertTrue(np.allclose(summandTwo, [1.7304,-5.1912,-2.5956]), 'valor incorrecto para summandTwo')

#### ** (3b) Usa los pesos para hacer predicciones **
#### A continuación implementa una función `getLabeledPredictions` que tome los pesos y una observación en formato  `LabeledPoint` y devuelva una tupla (label, prediction). Podremos predecir ahora calculando el dot product entre los pesos y las características de las observaciones.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def getLabeledPrediction(weights, observation):
    """ Calcula predicciones y devuelve una tupla (label, prediction).

    Nota:
        Las etiquetas deberían permanecer sin cambios ya que usaremos esta información para calcular el error
        de la predicción más tarde.

    Args:
        weights (np.ndarray): Un array con un peso por cada característica en `trainData`.
        observation (LabeledPoint): un `LabeledPoint` que contiene la etiqueta correcta y las características
        para el datapoint.
    Returns:
        tuple: una tupla (label, prediction).
    """
    return <RELLENA>

weights = np.array([1.0, 1.5])
predictionExample = sc.parallelize([LabeledPoint(2, np.array([1.0, .5])),
                                    LabeledPoint(1.5, np.array([.5, .5]))])
labelsAndPredsExample = predictionExample.map(lambda lp: getLabeledPrediction(weights, lp))
print labelsAndPredsExample.collect()

In [ ]:
# TEST Uso de pesos para hacer predicciones (3b)
Test.assertEquals(labelsAndPredsExample.collect(), [(2.0, 1.75), (1.5, 1.25)],
                  'incorrect definition for getLabeledPredictions')

#### ** (3c) Descenso de Gradiente **
#### A continuación implementa una función para la regresión lineal y pruébala en un ejemplo.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def linregGradientDescent(trainData, numIters):
    """ Calcula los pesos y el error para un modelo de regresión lineal entrenado con descenso de gradiente.

    Note:
       `DenseVector` se comporta de una manera similiar a `numpy.ndarray` y se pueden intercambiar dentro
        de esta función. Por ejemplo ambos implementan el método `dot`.

    Args:
        trainData (RDD of LabeledPoint): Los datos etiquetados para su uso en el modelo de training.
        numIters (int): Número de iteraciones que tiene que hacer el descenso de gradiente.

    Returns:
        (np.ndarray, np.ndarray): Una tupla de  (weights, training errors). Los pesos serán los pesos
        finales (un peso por característica) del model, y los errores de training contendrán un error (RMSE)
        para cada iteracion del algoritmo.
    """
    # cuenta de los datos de training
    n = trainData.count()
    # número de características en los datos de training
    d = len(trainData.take(1)[0].features)
    w = np.zeros(d)
    alpha = 1.0
    # Calcularemos y almacenaremos el error de training después de cada iteración 
    errorTrain = np.zeros(numIters)
    for i in range(numIters):
        # Usa getLabeledPrediction de (3b) con trainData para obtener un RDD de tuplas (label, prediction)
        # Los pesos son todos 0 para la primera iteración por lo que la predicción tendrá errores muy grandes
        # al principio.
        labelsAndPredsTrain = trainData.<RELLENA>
        errorTrain[i] = calcRMSE(labelsAndPredsTrain)

        # Calcula el `gradiente`. Utiliza la función `gradientSummand` de (3a).
        # `gradient` debería ser un `DenseVector` de longitud `d`.
        gradient = <RELLENA>

        # actualiza los pesos
        alpha_i = alpha / (n * np.sqrt(i+1))
        w -= <RELLENA>
    return w, errorTrain

# Crea un dataset de juguete con n = 10, d = 3 y lo ejecuta 5 iteraciones
# nota: El modelo resultante no es bueno pero el objetivo es verificar que 
# linregGradientDescent funciona
exampleN = 10
exampleD = 3
exampleData = (sc
               .parallelize(parsedTrainData.take(exampleN))
               .map(lambda lp: LabeledPoint(lp.label, lp.features[0:exampleD])))
print exampleData.take(2)
exampleNumIters = 5
exampleWeights, exampleErrorTrain = linregGradientDescent(exampleData, exampleNumIters)
print exampleWeights

In [ ]:
# TEST Descenso de gradiente (3c)
expectedOutput = [48.88110449,  36.01144093, 30.25350092]
Test.assertTrue(np.allclose(exampleWeights, expectedOutput), 'El valor de exampleWeights es incorrecto')
expectedError = [79.72013547, 30.27835699,  9.27842641,  9.20967856,  9.19446483]
Test.assertTrue(np.allclose(exampleErrorTrain, expectedError),
                'El valor de exampleErrorTrain es incorrecto')

#### ** (3d) Entrenar el model **
#### Vamos a entrenar un modelo de regresión lineal con todo nuestro dataset de training y evaluar su precisión contra el set de validación. No usaremos el set de test aqui ya que si evaluamos el modelo contra el set de test podriamos distorsionar los resultados finales.
#### Tenemos casi todo el trabajo hecho: Hemos calculado el número de características en la parte (1b), hemos creado los datasets de training y validación y calculado sus tamaños en la parte (1e) y hemos escrito una función que calcula el el RMSE  en la parte (2b).

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
numIters = 50
weightsLR0, errorTrainLR0 = linregGradientDescent(<RELLENA>)

labelsAndPreds = parsedValData.<RELLENA>
rmseValLR0 = calcRMSE(labelsAndPreds)

print 'Validación RMSE:\n\tBaseline = {0:.3f}\n\tLR0 = {1:.3f}'.format(rmseValBase,
                                                                       rmseValLR0)

In [ ]:
# TEST Entrenar el modelo (3d)
expectedOutput = [22.64535883, 20.064699, -0.05341901, 8.2931319, 5.79155768, -4.51008084,
                  15.23075467, 3.8465554, 9.91992022, 5.97465933, 11.36849033, 3.86452361]
Test.assertTrue(np.allclose(weightsLR0, expectedOutput), 'valores incorrecto para weightsLR0')

#### ** Visualización 4: Error de training **
#### Miraremos el logaritmo de error de entrenamiento (y) como una función de las iteraciones (x). El primer scatter plot visualiza esta función para las 50 primeras iteraciones. El segundo plot muestra el error en si mismo (sin logaritmo) viendo las últimas 44 iteraciones.

In [ ]:
norm = Normalize()
clrs = cmap(np.asarray(norm(np.log(errorTrainLR0))))[:,0:3]

fig, ax = preparePlot(np.arange(0, 60, 10), np.arange(2, 6, 1))
ax.set_ylim(2, 6)
plt.scatter(range(0, numIters), np.log(errorTrainLR0), s=14**2, c=clrs, edgecolors='#888888', alpha=0.75)
ax.set_xlabel('Iteration'), ax.set_ylabel(r'$\log_e(errorTrainLR0)$')
pass

In [ ]:
norm = Normalize()
clrs = cmap(np.asarray(norm(errorTrainLR0[6:])))[:,0:3]

fig, ax = preparePlot(np.arange(0, 60, 10), np.arange(17, 22, 1))
ax.set_ylim(17.8, 21.2)
plt.scatter(range(0, numIters-6), errorTrainLR0[6:], s=14**2, c=clrs, edgecolors='#888888', alpha=0.75)
ax.set_xticklabels(map(str, range(6, 66, 10)))
ax.set_xlabel('Iteration'), ax.set_ylabel(r'Training Error')
pass

### ** Parte 4: Entrenamiento usando MLib y ajuste de hiperparámetros via búsqueda en grid **

#### **(4a) `LinearRegressionWithSGD` **
#### Ya lo estamos haciendo mejor que el modelo de baseline pero veamos si podemos hacerlo aún mejor añadiendo un interceptor, usando regularización y (basándonos en las visualizaciones previas) entrenando para más iteraciones. La función de MLib [LinearRegressionWithSGD](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionWithSGD) implementa el mismo algoritmo que hemos hecho en la parte (3b) de una manera un poco más eficiente y con funcionalidad adicional como la aproximación del gradiente estocástico, inclusión de un interceptor en el modelo y regularizaciones L1 y L2. Primero usaremos LinearRegressionWithSGD para entrenar un modelo con regularización L2 y con un interceptor. Este método devuelve un modelo [LinearRegressionModel](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionModel).  A continuación usaremos los pesos del modelo  [weights](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionModel.weights) e  [intercept](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionModel.intercept) para imprimir los parámetros del modelo.

In [ ]:
from pyspark.mllib.regression import LinearRegressionWithSGD
# Valores a utilizar para entrenar el modelo de regresión lineal
numIters = 500  # iteraciones
alpha = 1.0  # paso (step)
miniBatchFrac = 1.0  # miniBatchFraction
reg = 1e-1  # parámetro de regularización (regParam) 
regType = 'l2'  # tipo de regularización (regType)
useIntercept = True  # interceptor

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
firstModel = LinearRegressionWithSGD.<RELLENA>

# weightsLR1 almacena los pesos del modelo ; interceptLR1 almacena los interceptores del modelo
weightsLR1 = <RELLENA>
interceptLR1 = <RELLENA>
print weightsLR1, interceptLR1

In [ ]:
# TEST LinearRegressionWithSGD (4a)
expectedIntercept = 13.3335907631
expectedWeights = [16.682292427, 14.7439059559, -0.0935105608897, 6.22080088829, 4.01454261926, -3.30214858535,
                   11.0403027232, 2.67190962854, 7.18925791279, 4.46093254586, 8.14950409475, 2.75135810882]
Test.assertTrue(np.allclose(interceptLR1, expectedIntercept), 'valor incorrecto para interceptLR1')
Test.assertTrue(np.allclose(weightsLR1, expectedWeights), 'valor incorrecto para weightsLR1')

#### **(4b) Predicción**
#### Ahora usaremos el método [LinearRegressionModel.predict()](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionModel.predict) para hacer predicciones sobre un punto de ejemplo. Pasa las  `features` de un `LabeledPoint` al método `predict()`.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
samplePoint = parsedTrainData.take(1)[0]
samplePrediction = <RELLENA>
print samplePrediction

In [ ]:
# TEST Predicción (4b)
Test.assertTrue(np.allclose(samplePrediction, 56.8013380112),
                'valor incorrecto para samplePrediction')

#### ** (4c) Evaluar RMSE **
#### A continuación vamos a evaluar la precisión de este modelo con el set de validación. Usa el método `predict()` para cear un RDD de  `labelsAndPreds` y entonces utiliza la función  `calcRMSE()` de la parte (2b).

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
labelsAndPreds = <RELLENA>
rmseValLR1 = <RELLENA>

print ('Validación RMSE:\n\tBaseline = {0:.3f}\n\tLR0 = {1:.3f}' +
       '\n\tLR1 = {2:.3f}').format(rmseValBase, rmseValLR0, rmseValLR1)

In [ ]:
# TEST Evaluación RMSE (4c)
Test.assertTrue(np.allclose(rmseValLR1, 19.691247), 'valor incorrecto para rmseValLR1')

#### ** (4d) Búsqueda en Grid **
#### Ya estamos superando como mínimo en 2 años la predicciones con respecto al baseline y con los datos del set de validación, veamos si podemos hacerlo todavía mejor. Vamos a realizar una búsqueda de un buen parámetro de regularización en un grid. vamos a probar `regParam` con los valores `1e-10`, `1e-5` y `1`.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
bestRMSE = rmseValLR1
bestRegParam = reg
bestModel = firstModel

numIters = 500
alpha = 1.0
miniBatchFrac = 1.0
for reg in <RELLENA>:
    model = LinearRegressionWithSGD.train(parsedTrainData, numIters, alpha,
                                          miniBatchFrac, regParam=reg,
                                          regType='l2', intercept=True)
    labelsAndPreds = parsedValData.map(lambda lp: (lp.label, model.predict(lp.features)))
    rmseValGrid = calcRMSE(labelsAndPreds)
    print rmseValGrid

    if rmseValGrid < bestRMSE:
        bestRMSE = rmseValGrid
        bestRegParam = reg
        bestModel = model
rmseValLRGrid = bestRMSE

print ('Validación RMSE:\n\tBaseline = {0:.3f}\n\tLR0 = {1:.3f}\n\tLR1 = {2:.3f}\n' +
       '\tLRGrid = {3:.3f}').format(rmseValBase, rmseValLR0, rmseValLR1, rmseValLRGrid)

In [ ]:
# TEST Búsqueda en grid (4d)
Test.assertTrue(np.allclose(17.017170, rmseValLRGrid), 'valor incorrecto para rmseValLRGrid')

#### ** Visualización 5: Mejor modelo de predicción**
#### Vamos a crear una visualización similar a la 'Visualización 3: Predichos vs. reales' de la parte 2 utilizando las prediciones del mejor modelo de la parte (4d) sobre el dataset de validación. En concreto, crearemos un scatter plot con códigos de color para visualizar tuplas que almacenan i) el valor predicho para este modelo y ii) la etiqueta verdadera.

In [ ]:
predictions = np.asarray(parsedValData
                         .map(lambda lp: bestModel.predict(lp.features))
                         .collect())
actual = np.asarray(parsedValData
                    .map(lambda lp: lp.label)
                    .collect())
error = np.asarray(parsedValData
                   .map(lambda lp: (lp.label, bestModel.predict(lp.features)))
                   .map(lambda (l, p): squaredError(l, p))
                   .collect())

norm = Normalize()
clrs = cmap(np.asarray(norm(error)))[:,0:3]

fig, ax = preparePlot(np.arange(0, 120, 20), np.arange(0, 120, 20))
ax.set_xlim(15, 82), ax.set_ylim(-5, 105)
plt.scatter(predictions, actual, s=14**2, c=clrs, edgecolors='#888888', alpha=0.75, linewidths=.5)
ax.set_xlabel('Predicted'), ax.set_ylabel(r'Actual')
pass

#### ** (4e) Cambiando alpha y el número de iteraciones **
#### En la búsqueda en grid previa establecimos  `alpha = 1` para todos los experimentos. Veamos ahora que sucede si variamos  `alpha`.  En concreto vamos a probar  `1e-5` y `10` como valores para `alpha` entrenando modelos con 500 iteracciones (como antes) pero también para 5 iteracciones. Evaluaremos todos los modelos con el dataset de validación. Si ponemos `alpha` muy pequeño el descenso del gradiente necesitará un gran número de pasos para converger a la solución y si utilizamos un `alpha` muy grande puede ocasionar problemas numéricos, como verás para `alpha = 10`.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
reg = bestRegParam
modelRMSEs = []

for alpha in <RELLENA>:
    for numIters in <RELLENA>:
        model = LinearRegressionWithSGD.train(parsedTrainData, numIters, alpha,
                                              miniBatchFrac, regParam=reg,
                                              regType='l2', intercept=True)
        labelsAndPreds = parsedValData.map(lambda lp: (lp.label, model.predict(lp.features)))
        rmseVal = calcRMSE(labelsAndPreds)
        print 'alpha = {0:.0e}, numIters = {1}, RMSE = {2:.3f}'.format(alpha, numIters, rmseVal)
        modelRMSEs.append(rmseVal)

In [ ]:
# TEST Variar alpha y el número de iteraciones(4e)
expectedResults = sorted([56.969705, 56.892949, 355124752.221221])
Test.assertTrue(np.allclose(sorted(modelRMSEs)[:3], expectedResults), 'valor incorrecto para modelRMSEs')

#### **Visualización 6: Mapa de calor de hiperparámetros **
#### A continuación realizaremos una visualización de búsqueda de hiperparámetros usando un conjunto grande de hiperparámetros precalculados. En concreto vamos a crear un mapa de calor donde los colores más brillantes se correponda con los valores más bajos de RMSE. El primer plot tiene un gran área de colores brillantes, para poder diferenciar dentro de esta región generamos un segundo plot con los hiperparámetros encontrados en ese área.

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Saved parameters and results, to save the time required to run 36 models
numItersParams = [10, 50, 100, 250, 500, 1000]
regParams = [1e-8, 1e-6, 1e-4, 1e-2, 1e-1, 1]
rmseVal = np.array([[  20.36769649,   20.36770128,   20.36818057,   20.41795354,  21.09778437,  301.54258421],
                    [  19.04948826,   19.0495    ,   19.05067418,   19.16517726,  19.97967727,   23.80077467],
                    [  18.40149024,   18.40150998,   18.40348326,   18.59457491,  19.82155716,   23.80077467],
                    [  17.5609346 ,   17.56096749,   17.56425511,   17.88442127,  19.71577117,   23.80077467],
                    [  17.0171705 ,   17.01721288,   17.02145207,   17.44510574,  19.69124734,   23.80077467],
                    [  16.58074813,   16.58079874,   16.58586512,   17.11466904,  19.6860931 ,   23.80077467]])

numRows, numCols = len(numItersParams), len(regParams)
rmseVal = np.array(rmseVal)
rmseVal.shape = (numRows, numCols)

fig, ax = preparePlot(np.arange(0, numCols, 1), np.arange(0, numRows, 1), figsize=(8, 7), hideLabels=True,
                      gridWidth=0.)
ax.set_xticklabels(regParams), ax.set_yticklabels(numItersParams)
ax.set_xlabel('Regularization Parameter'), ax.set_ylabel('Number of Iterations')

colors = LinearSegmentedColormap.from_list('blue', ['#0022ff', '#000055'], gamma=.2)
image = plt.imshow(rmseVal,interpolation='nearest', aspect='auto',
                    cmap = colors)

In [ ]:
# Zoom a abajo a la izquierda
numItersParamsZoom, regParamsZoom = numItersParams[-3:], regParams[:4]
rmseValZoom = rmseVal[-3:, :4]

numRows, numCols = len(numItersParamsZoom), len(regParamsZoom)

fig, ax = preparePlot(np.arange(0, numCols, 1), np.arange(0, numRows, 1), figsize=(8, 7), hideLabels=True,
                      gridWidth=0.)
ax.set_xticklabels(regParamsZoom), ax.set_yticklabels(numItersParamsZoom)
ax.set_xlabel('Regularization Parameter'), ax.set_ylabel('Number of Iterations')

colors = LinearSegmentedColormap.from_list('blue', ['#0022ff', '#000055'], gamma=.2)
image = plt.imshow(rmseValZoom,interpolation='nearest', aspect='auto',
                    cmap = colors)
pass

### ** Parte 5: Añadir interaciones entre características **

#### ** (5a) Añadir interacciones de dos vías **
#### Hasta ahora hemos utilizado las características tal y como nos venían dadas. Ahora vamos a añadir características que capturen interacciones de dos vías entre las características existentes (ver slides). Escribe una función `twoWayInteractions` que tome un `LabeledPoint` y genere un nuevo `LabeledPoint` que contenga las antiguas características y la interaciones de dos vías entre ellas. Por ejemplo un dataset con tres características tendría ahora nueve interacciones de dos vías  ( $ \scriptsize 3^2 $ ).
#### Puedes utilizar [itertools.product](https://docs.python.org/2/library/itertools.html#itertools.product) para genera tuplas por cada interacción de dos vías. Recuerda que puedes combinar dos objetos `DenseVector` o `ndarray` utilizando [np.hstack](http://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html#numpy.hstack).

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
import itertools

def twoWayInteractions(lp):
    """Crea un nuevo `LabeledPoint` que incluye interaciones de dos vias.

    Note:
        Para las características [x, y] las interaciones de dos vías serían [x^2, x*y, y*x, y^2] 
        y las añadiríamos a la lista [x, y] de características original.

    Args:
        lp (LabeledPoint): La etiquete y las características para esta observación
        
    Returns:
        LabeledPoint: La nueva `LabeledPoint` debería tener la misma etiqueta que `lp`. Sus características
        deberían incluir las caracteríticas de `lp` seguidas de las características de dos vías.
    """
    <RELLENA>

print twoWayInteractions(LabeledPoint(0.0, [2, 3]))

# Transforma los datasets de entrenamiento validación y test para incluir las interaciones de dos vías.
trainDataInteract = <RELLENA>
valDataInteract = <RELLENA>
testDataInteract = <RELLENA>

In [ ]:
# TEST añade interacciones de dos vías (5a)
twoWayExample = twoWayInteractions(LabeledPoint(0.0, [2, 3]))
Test.assertTrue(np.allclose(sorted(twoWayExample.features),
                            sorted([2.0, 3.0, 4.0, 6.0, 6.0, 9.0])),
                'Características incorrectas generadas por twoWayInteractions 1')
twoWayPoint = twoWayInteractions(LabeledPoint(1.0, [1, 2, 3]))
Test.assertTrue(np.allclose(sorted(twoWayPoint.features),
                            sorted([1.0,2.0,3.0,1.0,2.0,3.0,2.0,4.0,6.0,3.0,6.0,9.0])),
                'Características incorrectas generadas por twoWayInteractions 2')
Test.assertEquals(twoWayPoint.label, 1.0, 'Etiqueta incorrecta generada por twoWayInteractions')
Test.assertTrue(np.allclose(sum(trainDataInteract.take(1)[0].features), 40.821870576035529),
                'características incorrectas en trainDataInteract')
Test.assertTrue(np.allclose(sum(valDataInteract.take(1)[0].features), 45.457719932695696),
                'características incorrectas en valDataInteract')
Test.assertTrue(np.allclose(sum(testDataInteract.take(1)[0].features), 35.109111632783168),
                'características incorrectas en testDataInteract')

#### ** (5b) Construir el modelo de interacción **
#### Ahora vamos a construir el nuevo modelo. Para implementarlo con las nuevas características tenemos que cambiar algunos nombres de variables. Recuerda que deberíamos construir nuestro modelo con el dataset de training y evaluarlo con el de validación.

####  Debes de volver a ejecutar la búsqueda de hiperparámetros después de cambiar las características ya que los mejores hiperparámetros del modelo anterior no tienen porque ser necesariamente los mismos para el nuevo modelo. Para este ejercicio ya se han establecido los hiperparámetros a valores razonables.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
numIters = 500
alpha = 1.0
miniBatchFrac = 1.0
reg = 1e-10

modelInteract = LinearRegressionWithSGD.train(<RELLENA>, numIters, alpha,
                                              miniBatchFrac, regParam=reg,
                                              regType='l2', intercept=True)
labelsAndPredsInteract = <RELLENA>.map(lambda lp: (lp.label, <RELLENA>.predict(lp.features)))
rmseValInteract = calcRMSE(labelsAndPredsInteract)

print ('Validación RMSE:\n\tBaseline = {0:.3f}\n\tLR0 = {1:.3f}\n\tLR1 = {2:.3f}\n\tLRGrid = ' +
       '{3:.3f}\n\tLRInteract = {4:.3f}').format(rmseValBase, rmseValLR0, rmseValLR1,
                                                 rmseValLRGrid, rmseValInteract)

In [ ]:
# TEST Construir el modelo de interacción (5b)
Test.assertTrue(np.allclose(rmseValInteract, 15.6894664683), 'valor incorrecto para rmseValInteract')

#### ** (5c) Evaluar el modelo de interacción en los datos de test**
#### El último paso para evaluar el nuevo modelo es probarlo en el dataset de test. NO hemos utilizado aún el set de test para evaluar ninguno de los modelos. Así nuestra evaluacion nos proporcionará una estimación no sesgada de como se comportará el modelo con nuevos datos. Si hubiésemos cambiado nuestro modelo basándonos en como se hubiese comportado con el set de test nuestra estimacion de RMSE podría haber sido optimista.

#### También vamos a imprimir el RMSE para tanto para el modelo base como para el nuevo modelo, con esta información podemos ver como mejora el modelo con respecto al modelo base.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
labelsAndPredsTest = <RELLENA>
rmseTestInteract = <RELLENA>

print ('Test RMSE:\n\tBaseline = {0:.3f}\n\tLRInteract = {1:.3f}'
       .format(rmseTestBase, rmseTestInteract))

In [ ]:
# TEST Evalua el modelo de interacción sobre los datos de test(5c)
Test.assertTrue(np.allclose(rmseTestInteract, 16.3272040537),
                'valores incorrecto para rmseTestInteract')